# Creating Graph

In [ ]:
import os
import h5py
from sklearn.neighbors import NearestNeighbors
import numpy as np   
import ipyvolume as ipv
import time

import sys
sys.path.append("..") 
from cryoem.plots import plot_projection, plot_projections
from cryoem.projections import RotationMatrix

## Read generated data

Data was generated with:
```
python generator.py -mrc generated_data/bgal.mrc -num 5000
```

In [ ]:
# location of *.h5 files
data_dir = "../generated_data"

# half coverage (AngCoverage=0.5)
projections_filename = "ProjectionsAngles_ProjNber5000_AngCoverage0.5_AngShift1.57"

# load structures
data = h5py.File(os.path.join(data_dir, f"{projections_filename}.h5"), 'r')

## Add Gauss Noise to Projections

<div class="alert alert-warning" style="max-width: 20rem;">
  <h4 class="alert-heading">Add Noise</h4>
  <p class="mb-0">We start by adding Gaussian noise with variance=15, mean=0 in order to simulate the input data we will get to our pipeline.</p>
</div>

In [ ]:
from cryoem.noise import gaussian_noise

In [ ]:
projections = np.zeros(data["Projections"].shape)

In [ ]:
var = 100

In [ ]:
for datapoint in range(len(data["Projections"])):
    projections[datapoint] = data["Projections"][datapoint] + gaussian_noise(data["Projections"][datapoint].shape, mean=0, var=var)

## k-NN on **projections**

In [ ]:
projections.shape

In [ ]:
X = np.reshape(projections, (projections.shape[0], -1))
X.shape

In [ ]:
start_time = time.time()

if not os.path.exists(f'data/{projections_filename}_WithGaussNoise{var}_distances.npy'):
    nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)
    connections = nbrs.kneighbors_graph(X).toarray()
    
    np.save(f'data/{projections_filename}_WithGaussNoise{var}_indices', indices)         # Indices of the nearest points in the population matrix
    np.save(f'data/{projections_filename}_WithGaussNoise{var}_distances', distances)     # Array representing the lengths to points
    np.save(f'data/{projections_filename}_WithGaussNoise{var}_connections', connections) # Sparse graph showing the connections between neighboring points
    
else:
    indices     = np.load(f'data/{projections_filename}_WithGaussNoise{var}_indices.npy')     # shape: NUM_IMGS, NUM_NEIGHBOURS
    distances   = np.load(f'data/{projections_filename}_WithGaussNoise{var}_distances.npy')   # shape: NUM_IMGS, NUM_NEIGHBOURS
    connections = np.load(f'data/{projections_filename}_WithGaussNoise{var}_connections.npy') # shape: NUM_IMGS, NUM_IMGS
    
print(f"--- {time.time() - start_time} seconds ---")

In [ ]:
connections.shape

In [ ]:
indices[0]

In [ ]:
distances[0]

### Plot similar projections

In [ ]:
indices1 = indices[0]
distances1 = distances[0]

nrows, ncols = 1, 5
start_row, start_col = 0, 0

images = []
titles = []

cr = [(i, j) for i in range(nrows) for j in range(ncols)]

for datapoint, distance in zip(indices1, distances1):
    image = projections[datapoint]
    image = image #+ gaussian_noise(shape=image.shape, mean=0, var=0)

    angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
    title = f'Projection {datapoint}\nAngles {angles}\nDistance {distance:.2f}'
    
    images.append(image)
    titles.append(title)
        
plot_projections(images, titles, nrows=nrows, ncols=ncols)

In [ ]:
indices1 = indices[10]
distances1 = distances[10]

nrows, ncols = 1, 5
start_row, start_col = 0, 0

images = []
titles = []

cr = [(i, j) for i in range(nrows) for j in range(ncols)]

for datapoint, distance in zip(indices1, distances1):
    image = projections[datapoint]
    image = image #+ gaussian_noise(shape=image.shape, mean=0, var=0)

    angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
    title = f'Projection {datapoint}\nAngles {angles}\nDistance {distance:.2f}'
    
    images.append(image)
    titles.append(title)
        
plot_projections(images, titles, nrows=nrows, ncols=ncols)

##### Debug angles

In [ ]:
data["Angles"][0]

In [ ]:
min(data["Angles"][:, 0]), max(data["Angles"][:, 0]), np.pi/2, 2*np.pi - np.pi/2

#### Without Rotation Matrix

In [ ]:
indices1 = indices[1]

knn_vectors = np.take(data["Angles"], indices1, axis=0)
all_vectors = np.delete(data["Angles"], indices1, 0)

ipv.figure(width=500, height=400)
ipv.scatter(knn_vectors[:,0], knn_vectors[:,2], knn_vectors[:,1], marker="diamond", color="red", size=1.5)
ipv.scatter(all_vectors[:,0], all_vectors[:,2], all_vectors[:,1], marker="sphere", color="blue", size=1)
ipv.show()

#### With Rotation Matrix

In [ ]:
indices1 = indices[1000]

knn_vectors = RotationMatrix(np.take(data["Angles"], indices1, axis=0))
all_vectors = RotationMatrix(np.delete(data["Angles"], indices1, 0))

print(all_vectors.shape)
print(all_vectors[0])

ipv.figure(width=500, height=400)
ipv.scatter(knn_vectors[:,0], knn_vectors[:,2], knn_vectors[:,1], marker="diamond", color="red", size=1.5)
ipv.scatter(all_vectors[:,0], all_vectors[:,2], all_vectors[:,1], marker="sphere", color="blue", size=1)
ipv.show()

---

# Quaternions

Visualizing **SO(3) space**. Following explaination in [wiki](https://en.wikipedia.org/wiki/3D_rotation_group#Topology).

How to use [pyquaternions](http://kieranwynn.github.io/pyquaternion/).


In [ ]:
from pyquaternion import Quaternion

## One Quaternion

In [ ]:
angle1 = data["Angles"][0]
angle1

In [ ]:
qx = Quaternion(axis=[1, 0, 0], angle=angle1[0])
qy = Quaternion(axis=[0, 1, 0], angle=angle1[1])
qz = Quaternion(axis=[0, 0, 1], angle=angle1[2])

In [ ]:
# compose rotations above
q = qx*qy*qz
q.axis, q.angle

In [ ]:
point = list(map(lambda x: np.array([x * q.angle]), q.axis))
point

## All Quaternions

In [ ]:
def quaternion(angle):
    """
    Quaternion implements 3 rotations along x, y, z axis. 
    We compose them to get the final (single) rotation.
    """
    qx = Quaternion(axis=[1, 0, 0], angle=angle[0])
    qy = Quaternion(axis=[0, 1, 0], angle=angle[1])
    qz = Quaternion(axis=[0, 0, 1], angle=angle[2])
    
    # compose rotations above
    q = qx*qy*qz

    return q

In [ ]:
def quaternion2point(q):
    """ Convert Quaternion to point
    
    We convert Qaternion to the point described with x, y, z values in the Cartesian coordinate system.
    From the Qaternion we get axis and angle. The axis is described as unit vector (ux, uy, uz) and the angle is magnitude of vector.
    Using this two information, we can get the x, y, z coordinates of the point described with axis and angle.
    """
    point = np.array(list(map(lambda x: x * q.angle, q.axis)))
    return point

In [ ]:
points1 = np.zeros(data["Angles"].shape)

for idx, angles in enumerate(data["Angles"]):
    point = quaternion2point(quaternion(angles))
    points1[idx, :] = point

points1

In [ ]:
ipv.figure(width=500, height=400)
ipv.scatter(points1[:, 0], points1[:, 1], points1[:, 2], marker="sphere", color="blue", size=1.5)
ipv.show()

##### Debug the SO(3) space

In [ ]:
max(points1[:,0]), min(points1[:,0])

---

# kNN results in the SO(3)

## Half coverage angles

For kNN calculated on the half coverage angles, we have the following output of the kNN that was estimated on those projections.

In [ ]:
indices1 = indices[10]

knn_vectors = np.take(points1, indices1, axis=0)
all_vectors = np.delete(points1, indices1, 0)

ipv.figure(width=500, height=400)
ipv.scatter(knn_vectors[:,0], knn_vectors[:,2], knn_vectors[:,1], marker="diamond", color="red", size=1.5)
ipv.scatter(all_vectors[:,0], all_vectors[:,2], all_vectors[:,1], marker="sphere", color="blue", size=1)
ipv.show()

# Graph visualization

You can use the [PyGSP](https://github.com/epfl-lts2/pygsp) (a python package developed by our lab) to
look at it like a graph. 

Install with: 
```
pip install git+https://github.com/epfl-lts2/pygsp@naspert-nn_refactor. 
```

Then build the graph with graph = pygsp.graphs.Graph(adjacency, coords=embedding),
where adjacency is the kNN matrix, i.e., adjacency[i, j] is the
similarity between nodes i and j  
(you can make it binary, that is **=1** if
node j is one of the k nearest neighbors of node i, **=0** otherwise), and
embedding[i] is the coordinate of node i in that 3D space.

You can also build the kNN graph directly with the PyGSP as graph =
pygsp.graphs.NNGraph(X), where X is the images in pixel space. That is,
the same matrix you're passing to sklearn.neighbors.NearestNeighbors.
Then set the 3D coordinates with graph.coords = embedding. Ideally, we
should integrate sklearn as another backend and allow NNGraph to accept
a custom distance function.

Doing the above should give you intuitions about a graph embedded in SO(3).

In [ ]:
np.fill_diagonal(connections, 0)

In [ ]:
indices

In [ ]:
distances

In [ ]:
connections

In [ ]:
import pygsp

In [ ]:
adjacency = connections  # kNN matrix, 1 if nodes i and j are neighbours, 0 othervise
embedding = points1      # SO(3) space coordinates

print(f"Adjacency shape: {adjacency.shape}")
print(f"Embedding shape: {embedding.shape}")

graph = pygsp.graphs.Graph(adjacency, coords=embedding)
fig, ax = graph.plot(edge_width=2, edge_color='black')
fig.set_figwidth(20)
fig.set_figheight(20)

In [ ]:
adjacency = connections  # kNN matrix, 1 if nodes i and j are neighbours, 0 othervise
embedding = RotationMatrix(data["Angles"])[:,0:3] #points1 

print(f"Adjacency shape: {adjacency.shape}")
print(f"Embedding shape: {embedding.shape}")

graph = pygsp.graphs.Graph(adjacency, coords=embedding)
fig, ax = graph.plot(edge_width=20, edge_color='black')
fig.set_figwidth(20)
fig.set_figheight(20)

---